<a href="https://colab.research.google.com/github/falawar7/AAI_635O/blob/main/Week2/FE_Lesson_4Hybrid_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preparation**

## **1) Import Libraries**

In [2]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831161 sha256=8f9966e064ee637b5e8422df281265bf5ee6ef924eeff71be314e056e60a8421
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


## **2) Load Data**

In [3]:
#import Libraries
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
import time
import sklearn
from sklearn import model_selection

In [12]:
order_df = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx', 'order')
customer_df = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx', 'customer')
product_df = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx', 'product')

In [13]:
order_df

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850
...,...,...,...,...,...,...,...,...,...
272399,569629,23399,12,2011-10-05 11:37:00,2011-10-08 11:37:00,0.11,Delivery Truck,5.81,15249
272400,569629,22727,4,2011-10-05 11:37:00,2011-10-08 11:37:00,0.26,Delivery Truck,5.81,15249
272401,569629,23434,12,2011-10-05 11:37:00,2011-10-08 11:37:00,0.42,Delivery Truck,5.81,15249
272402,569629,23340,12,2011-10-05 11:37:00,2011-10-07 11:37:00,0.08,Regular Air,15.22,15249


In [14]:
customer_df

,CustomerID,Gender,Age,Income,Zipcode,Customer Segment
0,13089,male,53,High,8625,Small Business
1,15810,female,22,Low,87797,Small Business
2,15556,female,29,High,29257,Corporate
3,13137,male,29,Medium,97818,Middle class
4,16241,male,36,Low,79200,Small Business
...,...,...,...,...,...,...
4367,17763,female,22,High,57980,Corporate
4368,16078,male,23,High,38622,Middle class
4369,13270,male,40,Medium,57985,Middle class
4370,15562,male,37,Medium,91274,Middle class


In [15]:
product_df

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99
...,...,...,...,...,...,...
29907,NaN,NaN,NaN,Clothing|Men|Mens T-Shirts & Tank Tops|Mens T-...,Tee Bangers,26.99
29908,NaN,NaN,NaN,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.99
29909,NaN,NaN,NaN,Clothing|Women|Womens Dresses & Jumpsuits|Wome...,Skylinewears,35.99
29910,NaN,NaN,NaN,Office|Envelopes & Mailing Supplies|Envelopes,Envelopes.com,446.31


## **3) Merge Data**

In [16]:
# Merge the data
merged_df = pd.merge(order_df, customer_df, left_on=['CustomerID'], right_on=['CustomerID'], how='left')
merged_df = pd.merge(merged_df, product_df, left_on=['StockCode'], right_on=['StockCode'], how='left')

In [17]:
merged_df

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID,Gender,Age,Income,Zipcode,Customer Segment,Product Name,Description,Category,Brand,Unit Price
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,"3 1/2""W x 20""D x 20""H Funston Craftsman Smooth...",Our Rustic Collection is an instant classic. O...,Home Improvement|Hardware|Brackets and Angle I...,Ekena Milwork,199.11
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,Awkward Styles Shamrock Flag St. Patrick's Day...,Our St Patrick's Day Collection is perfect for...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens Gr...,Awkward Styles,23.95
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,Ebe Men Black Rectangle Half Rim Spring Hinge ...,Count on EBE for all of your eye correction ne...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,26.99
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,MightySkins Skin Decal Wrap Compatible with Ap...,Mightyskins are removable vinyl skins for prot...,Electronics|Electronics Learning Center|Ads Fr...,Mightyskins,14.99
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850,female,48,Medium,84306,Middle class,awesome since 1948 - 69th birthday gift t-shir...,awesome since 1948 - 69th birthday gift t-shir...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens T-...,Shirtinvaders,49.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272399,569629,23399,12,2011-10-05 11:37:00,2011-10-08 11:37:00,0.11,Delivery Truck,5.81,15249,male,36,High,7022,Corporate,MightySkins Skin Decal Wrap Compatible with YE...,Mightyskins are removable vinyl skins for prot...,Home|Kitchen & Dining|Dining & Entertaining|Dr...,Mightyskins,9.99
272400,569629,22727,4,2011-10-05 11:37:00,2011-10-08 11:37:00,0.26,Delivery Truck,5.81,15249,male,36,High,7022,Corporate,"Gillette Sensor Refill Blade Cartridges, 10 Co...",Gillette Sensor Men's Razor Blade Refills: Sel...,Personal Care|Shaving|Razors,Gillette,14.74
272401,569629,23434,12,2011-10-05 11:37:00,2011-10-08 11:37:00,0.42,Delivery Truck,5.81,15249,male,36,High,7022,Corporate,Ebe Women Reading Glasses Reader Cheaters Anti...,Stunning in coffee brown we show these specs w...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,23.99
272402,569629,23340,12,2011-10-05 11:37:00,2011-10-07 11:37:00,0.08,Regular Air,15.22,15249,male,36,High,7022,Corporate,MightySkins Skin Decal Wrap Compatible with Sm...,Do You Want Your Smok G150 To Look Different T...,Electronics|Electronics Learning Center|Ads Fr...,Mightyskins,5.99


# **Data Preprocessing**

### **4) Prepare Data**

**Step 1: Get Unique Users and Products**

**We create functions to get the unique list of users and products.**

In [18]:
# Get Unique Users and Product
def unique_users(data, column):
         return np.sort(data[column].unique())

def unique_items(data, column):
         return data[column].unique()

user_list = unique_users(order_df, "CustomerID")
item_list = unique_items(product_df, "Product Name")

print(user_list)  # Output: unique user IDs
print(item_list)  # Output: unique product names

[12346 12347 12348 ... 18282 18283 18287]
['Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover'
 'Eye Buy Express Prescription Glasses Mens Womens Burgundy Crystal Clear Yellow Rounded Rectangular Reading Glasses Anti Glare grade'
 "MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options"
 'Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches'
 'Stupell Industries Chevron Initial Wall D cor'
 "MightySkins Skin Decal Wrap Compatible with OtterBox Sticker Protective Cover 100's of Color Options"
 "MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options"
 'MightySkins Skin For OtterBox Commuter iPhone 8 Plus | Protective, Durable, and Unique Vinyl Decal wrap cover | Easy To Apply, Remove, and Change Styles | Made in the USA'
 "Dr. Comfort Paradise Women's Casual Shoe: 4.5 X-Wide (E-2E) Black Velcro"
 "MightySkins Skin Decal Wrap Co

**Step 2: Get Unique Feature Values**
**We combine features such as Customer Segment, Age, and Gender into one list.**

In [19]:
def features_to_add(customer, column1, column2, column3):
        return pd.concat([customer[column1], customer[column2], customer[column3]], ignore_index=True).unique()

feature_unique_list = features_to_add(customer_df, 'Customer Segment', "Age", "Gender")
print(feature_unique_list)  # Output: unique feature values

['Small Business' 'Corporate' 'Middle class' 53 22 29 36 48 45 47 23 39 34
 52 51 35 19 26 37 18 20 21 41 31 28 50 38 30 25 32 55 43 54 49 40 33 44
 46 42 27 24 'male' 'female']


**Step 3: Create ID Mappings**

**We convert user, item, and feature IDs into integer indices.**

In [22]:
def mapping(user_list, item_list, feature_unique_list):
  user_to_index_mapping = {user_id: idx for idx, user_id in enumerate(user_list)}
  item_to_index_mapping = {item_id: idx for idx, item_id in enumerate(item_list)}
  feature_to_index_mapping = {feature_id: idx for idx, feature_id in enumerate(feature_unique_list)}

  return user_to_index_mapping, item_to_index_mapping, feature_to_index_mapping

user_to_index_mapping, item_to_index_mapping, feature_to_index_mapping = mapping(user_list, item_list, feature_unique_list)
print(user_to_index_mapping)  # Output: user-to-index mapping

{np.int64(12346): 0, np.int64(12347): 1, np.int64(12348): 2, np.int64(12350): 3, np.int64(12352): 4, np.int64(12353): 5, np.int64(12354): 6, np.int64(12355): 7, np.int64(12356): 8, np.int64(12358): 9, np.int64(12359): 10, np.int64(12360): 11, np.int64(12361): 12, np.int64(12362): 13, np.int64(12363): 14, np.int64(12364): 15, np.int64(12365): 16, np.int64(12370): 17, np.int64(12372): 18, np.int64(12373): 19, np.int64(12375): 20, np.int64(12377): 21, np.int64(12378): 22, np.int64(12379): 23, np.int64(12380): 24, np.int64(12381): 25, np.int64(12383): 26, np.int64(12384): 27, np.int64(12386): 28, np.int64(12388): 29, np.int64(12390): 30, np.int64(12393): 31, np.int64(12394): 32, np.int64(12395): 33, np.int64(12397): 34, np.int64(12399): 35, np.int64(12401): 36, np.int64(12402): 37, np.int64(12405): 38, np.int64(12406): 39, np.int64(12407): 40, np.int64(12408): 41, np.int64(12409): 42, np.int64(12410): 43, np.int64(12412): 44, np.int64(12413): 45, np.int64(12414): 46, np.int64(12415): 47, n

# **Interaction Matrices**

## **5) Build the Recommendation System**

**Step 1: User-to-Product Relationship**

**We calculate the total quantity per user-product.**

In [23]:
user_to_product = merged_df[['CustomerID', 'Product Name', 'Quantity']]
user_to_product = user_to_product.groupby(['CustomerID', 'Product Name']).agg({'Quantity': 'sum'}).reset_index()
user_to_product.tail()

,CustomerID,Product Name,Quantity
138397,18287,Sport-Tek Ladies PosiCharge Competitor Tee,24
138398,18287,Ultra Sleek And Spacious Pearl White Lacquer 1...,6
138399,18287,"Union 3"" Female Ports Stainless Steel Pipe Fit...",12
138400,18287,awesome since 1948 - 69th birthday gift t-shir...,4
138401,18287,billyboards Porcelain Menu Chalkboard,6


**Step 2: Product-to-Features Relationship**

**Next, we calculate the total quantity per customer segment and product.**

In [24]:
#product to feature
product_to_feature = merged_df[['Product Name', 'Customer Segment', 'Quantity']]
product_to_feature = product_to_feature.groupby(['Product Name', 'Customer Segment']).agg({'Quantity': 'sum'}).reset_index()
product_to_feature.head()

,Product Name,Customer Segment,Quantity
0,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Corporate,712
1,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Middle class,272
2,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Small Business,388
3,"""Soccer"" Vinyl Graphic - Large - Ivory",Corporate,1940
4,"""Soccer"" Vinyl Graphic - Large - Ivory",Middle class,1418


**Step 3: Split Data into Training and Testing**

**We split the user-product relationship data into training and testing datasets.**

In [25]:
# split the user-product relationship data into training and testing datasets
user_to_product_train, user_to_product_test = model_selection.train_test_split(user_to_product, test_size=0.33, random_state=42)
print("Training set size:")
print(user_to_product_train.shape)
print("Test set size:")
print(user_to_product_test.shape)


Training set size:
(92729, 3)
Test set size:
(45673, 3)


**Step 4: Create the Interaction Matrix**

**We create the interaction matrix, where the value represents the quantity.**

In [26]:
#create the interaction Matrix , value represent the Quanity

def interactions(data, row, col, value, row_map, col_map):
        row = data[row].apply(lambda x: row_map[x]).values
        col = data[col].apply(lambda x: col_map[x]).values
        value = data[value].values
        return coo_matrix((value, (row, col)), shape=(len(row_map), len(col_map)))
user_to_product_interaction_train = interactions(user_to_product_train, "CustomerID", "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)
user_to_product_interaction_test = interactions(user_to_product_test, "CustomerID", "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)
print(user_to_product_interaction_train)

<COOrdinate sparse matrix of dtype 'int64'
	with 92729 stored elements and shape (3647, 797)>
  Coords	Values
  (2124, 230)	10
  (1060, 268)	16
  (1633, 295)	3
  (113, 45)	20
  (3100, 44)	54
  (3110, 25)	4
  (2133, 126)	6
  (525, 176)	2
  (2711, 45)	26
  (281, 487)	25
  (1378, 349)	27
  (2356, 18)	12
  (3248, 6)	24
  (1295, 339)	1
  (1229, 4)	120
  (3153, 36)	24
  (2441, 54)	1
  (1522, 124)	4
  (1436, 15)	8
  (3613, 137)	12
  (265, 324)	1
  (3309, 220)	24
  (2943, 3)	18
  (2021, 407)	5
  (2516, 187)	18
  :	:
  (64, 8)	24
  (3406, 109)	1
  (3219, 12)	12
  (2196, 96)	25
  (3409, 236)	12
  (2216, 172)	1
  (2720, 60)	5
  (121, 326)	6
  (1670, 63)	31
  (1555, 4)	6
  (22, 112)	12
  (1670, 8)	15
  (1736, 41)	10
  (1101, 256)	8
  (415, 152)	12
  (3303, 154)	2
  (2955, 8)	16
  (2276, 27)	1
  (3612, 527)	12
  (1446, 299)	192
  (2888, 238)	60
  (3140, 35)	2
  (2727, 591)	7
  (3451, 194)	1
  (3203, 15)	4


**Step 5: Product-to-Features Interaction Matrix**

**We create the product-to-features interaction matrix similarly.**

In [27]:
product_to_feature_interaction = interactions(product_to_feature, "Product Name", "Customer Segment", "Quantity", item_to_index_mapping, feature_to_index_mapping)

In [29]:
print(product_to_feature_interaction)

<COOrdinate sparse matrix of dtype 'int64'
	with 2083 stored elements and shape (797, 43)>
  Coords	Values
  (262, 1)	712
  (262, 2)	272
  (262, 0)	388
  (167, 1)	1940
  (167, 2)	1418
  (167, 0)	2797
  (482, 1)	233
  (482, 2)	428
  (482, 0)	303
  (399, 1)	38
  (399, 2)	24
  (399, 0)	4
  (676, 1)	6
  (676, 0)	18
  (379, 1)	21
  (379, 2)	8
  (379, 0)	31
  (409, 2)	1
  (478, 1)	108
  (478, 2)	78
  (478, 0)	89
  (527, 1)	1266
  (527, 2)	340
  (527, 0)	612
  (156, 1)	7562
  :	:
  (198, 2)	1521
  (198, 0)	1231
  (199, 1)	69
  (199, 2)	87
  (199, 0)	109
  (76, 1)	9422
  (76, 2)	8254
  (76, 0)	10122
  (12, 1)	1648
  (12, 2)	1822
  (12, 0)	1754
  (623, 1)	8
  (623, 2)	3
  (623, 0)	4
  (326, 1)	76
  (326, 2)	103
  (326, 0)	124
  (551, 1)	139
  (551, 2)	96
  (551, 0)	164
  (278, 2)	14
  (278, 0)	1
  (378, 1)	682
  (378, 2)	1603
  (378, 0)	494


# **Model Building With LightFM**

## **Example: Loss Function = “WARP”**

We train the model using the “WARP” loss function. WARP stands for Weighted Approximate-Rank Pairwise. It is designed to optimize the ranking performance of the recommendation system by minimizing the ranking error. It focuses on improving the rank of relevant items by treating misranked pairs of items as errors. WARP considers the distance between the positive and negative items in the ranking.

WARP loss function is used when you want to optimize for better ranking quality. It is useful for implicit feedback systems where the goal is to get items ranked as high as possible in the recommendation list.

In [48]:
#Build LightFM Model with features
model_with_features = LightFM(loss="warp")
model_with_features.fit_partial(user_to_product_interaction_train,
item_features=product_to_feature_interaction,
epochs=1,
num_threads=4, verbose=False)


In [49]:
#Calculate the AUC score
auc_with_features = auc_score(model=model_with_features,
                              test_interactions=user_to_product_interaction_test,
                              train_interactions=user_to_product_interaction_train,
                              item_features=product_to_feature_interaction,
                              num_threads=4)
print("AUC = {0:.2f}".format(auc_with_features.mean()))

AUC = 0.16


## **Logistic Loss**

This loss function is used for binary classification tasks, where interactions are either positive or negative (e.g., a user either clicked on a product or did not). It treats the recommendation problem as a series of binary decisions.



*   **When to use:** Use this loss function when your data consists of implicit feedback (e.g., clicks, views, purchases) that is binary in nature (yes/no).
*   **Effective for**: Implicit feedback datasets where the goal is to predict the likelihood of a user interacting with an item.





In [37]:
#Build LightFM Model with features using Logistic Loss
model_with_features = LightFM(loss="logistic")
model_with_features.fit_partial(user_to_product_interaction_train,
item_features=product_to_feature_interaction,
epochs=1,
num_threads=4, verbose=False)

In [38]:
#Calculate the AUC score for Logistic Loss
auc_with_features = auc_score(model=model_with_features,
                              test_interactions=user_to_product_interaction_test,
                              train_interactions=user_to_product_interaction_train,
                              item_features=product_to_feature_interaction,
                              num_threads=4)
print("AUC = {0:.2f}".format(auc_with_features.mean()))

AUC = 0.89


## **Bayesian Personalized Ranking (BPR)**

BPR is designed to maximize the ranking of relevant items over irrelevant ones. It works by comparing a positive item (an item that the user interacted with) to a randomly selected negative item, aiming to make the positive item more likely to be ranked higher than the negative item.


*   **When to use:** Use BPR when the goal is to learn a ranking of items, especially when dealing with implicit feedback. It's useful in recommendation systems where the relative ranking of items is more important than predicting exact ratings or interactions.
*   **Effective for:** Implicit feedback datasets, especially when the goal is to rank items rather than predict binary interactions.



In [39]:
#Build LightFM Model with features using BPR
model_with_features = LightFM(loss="bpr")
model_with_features.fit_partial(user_to_product_interaction_train,
item_features=product_to_feature_interaction,
epochs=1,
num_threads=4, verbose=False)

In [40]:
#Calculate the AUC score for BPR
auc_with_features = auc_score(model=model_with_features,
                              test_interactions=user_to_product_interaction_test,
                              train_interactions=user_to_product_interaction_train,
                              item_features=product_to_feature_interaction,
                              num_threads=4)
print("AUC = {0:.2f}".format(auc_with_features.mean()))

AUC = 0.16


# **Getting Recommendations**

In [42]:
def get_recommendations(model, user, items, user_to_product_interaction_matrix,
                        user2index_map, product_to_feature_interaction_matrix):
    # getting the user index
    userindex = user2index_map.get(user, None)
    if userindex == None:
        return None
    users = userindex

    # getting products already bought
    known_positives = items[user_to_product_interaction_matrix.tocsr()[userindex].indices]
    print('User index =', users)

    # scores from model prediction
    scores = model.predict(user_ids=users,
                           item_ids=np.arange(user_to_product_interaction_matrix.shape[1]),
                           item_features=product_to_feature_interaction_matrix)

    # getting top items
    top_items = items[np.argsort(-scores)]

    # printing out the result
    print("User %s" % user)
    print(" Known positives:")
    for x in known_positives[:10]:
        print(" %s" % x)
    print(" Recommended:")
    for x in top_items[:10]:
        print(" %s" % x)

In [50]:
#get reccomenations
get_recommendations(model_with_features, 17017, item_list, user_to_product_interaction_train, user_to_index_mapping, product_to_feature_interaction)

User index = 2888
User 17017
 Known positives:
 Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover
 MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options
 MightySkins Skin Decal Wrap Compatible with Lenovo Sticker Protective Cover 100's of Color Options
 Union 3" Female Ports Stainless Steel Pipe Fitting
 Ebe Women Reading Glasses Reader Cheaters Anti Reflective Lenses TR90 ry2209
 3 1/2"W x 32"D x 36"H Traditional Arts & Crafts Smooth Bracket, Douglas Fir
 MightySkins Skin Decal Wrap Compatible with HP Sticker Protective Cover 100's of Color Options
 MightySkins Skin Decal Wrap Compatible with Apple Sticker Protective Cover 100's of Color Options
 New Way 1120 - Crewneck Your Job Could Be Worse Toilet Paper Sweatshirt 4XL Light Pink
 3 1/2"W x 20"D x 20"H Funston Craftsman Smooth Bracket, Douglas Fir
 Recommended:
 nan
 Home Cardboard Flower Print Travel Memo Collection Picture Book Photo Album Red
 3 Pack - Re